# Credit Card Fraud Prediction

In this kernel I will do some explorations trying to understand the fraud transaction patterns and them I will implement some models of machine learning.<br>

I will implement technique an technique called SMOTE, supervised models, semi supervised learning algorithms and a deep learning model.

# About The Dataset

> The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.
> 
> It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.



> Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.


> The dataset has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. More details on current and past projects on related topics are available on http://mlg.ulb.ac.be/BruFence and http://mlg.ulb.ac.be/ARTML
>
> Please cite: Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

## Main Objectives

To read this dataset, balance it and use either one or more of modeling algorithms to predict accuracy of test dataset.

# Importing the libraries and looking at the data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec


from imblearn.pipeline import make_pipeline as make_pipeline_imb # To do our transformation in a unique time
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import make_pipeline
from imblearn.metrics import classification_report_imbalanced

from sklearn.model_selection import train_test_split
from collections import Counter

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import precision_score, recall_score, fbeta_score, confusion_matrix, precision_recall_curve, accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#loading the data
data = pd.read_csv("../input/creditcard.csv")

In [ ]:
data.sample(10)

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data[["Time","Amount","Class"]].describe()

In [ ]:
# Lets start looking the difference by Normal and Fraud transactions
data["Class"].value_counts()

In [ ]:
plt.figure(figsize=(7,5))
sns.countplot(data['Class'])
plt.title("Class Histogram", fontsize=18)
plt.xlabel("Class", fontsize=15)
plt.ylabel("Count", fontsize=15)
plt.show()

We have a clearly imbalanced data which it should be when treating of frauds.

## Looking the statistics of our Amount class frauds and normal transactions

In [ ]:
#To clearly the data of frauds and no frauds
df_fraud = data[data['Class'] == 1]
df_normal = data[data['Class'] == 0]

print("Fraud transaction statistics")
print(df_fraud["Amount"].describe())
print("\nNormal transaction statistics")
print(df_normal["Amount"].describe())

In [ ]:
#Feature engineering to a better visualization of the values
data['Amount_log'] = np.log(data.Amount + 0.01)

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(121)
ax = sns.boxplot(x ="Class",y="Amount", data=data)
ax.set_title("Class x Amount", fontsize=20)
ax.set_xlabel("Is Fraud?", fontsize=16)
ax.set_ylabel("Amount(US)", fontsize = 16)

plt.subplot(122)
ax1 = sns.boxplot(x ="Class",y="Amount_log", data=data)
ax1.set_title("Class x Amount", fontsize=20)
ax1.set_xlabel("Is Fraud?", fontsize=16)
ax1.set_ylabel("Amount(Log)", fontsize = 16)

plt.subplots_adjust(hspace = 0.6, top = 0.8)

plt.show()

We can see a slightly difference in log amount of our two Classes. <br>
The IQR of fraudulent transactions are higher than normal transactions, but normal transactions have highest values

## Data Exploration - Time
As our Time feature are in seconds, we will transform it to minutes and hours to get a better understanding of any underlying patterns

In [ ]:
timedelta = pd.to_timedelta(data['Time'], unit='s')
data['Time_hour'] = (timedelta.dt.components.hours).astype(int)

In [ ]:
plt.figure(figsize=(20,8))
sns.distplot(data[data['Class'] == 0]["Time_hour"], color='g')
sns.distplot(data[data['Class'] == 1]["Time_hour"],  color='r')
plt.title('Fraud x Normal Transactions by Hours', fontsize=17)
plt.xlim([-1,25])
plt.show()

No clear pattern, so nothing we can do about the Time column in the dataset.

In [ ]:
ax = sns.lmplot(y="Amount", x="Time_hour", fit_reg=False,aspect=1.8, data=data, hue='Class')
plt.title("Amount x Hour of Fraud, Normal Transactions", fontsize=16)

plt.show()

# EDA using boxplots
- Since we don't know what each feature is, we will create boxplots of all the V featues.
- We are searching for features that diverges from normal distribuition

In [ ]:
#Looking the V's features
columns = data.iloc[:,1:29].columns

frauds = data.Class == 1
normals = data.Class == 0

grid = gridspec.GridSpec(10, 3)
plt.figure(figsize=(15,20*4))

for n, col in enumerate(data[columns]):
    ax = plt.subplot(grid[n])
    sns.distplot(data[col][frauds], bins = 50, color='g') #Will receive the "semi-salmon" violin
    sns.distplot(data[col][normals], bins = 50, color='r') #Will receive the "ocean" color
    ax.set_ylabel('Density')
    ax.set_title(str(col))
    ax.set_xlabel('')
plt.show()

# Feature Engineering

In [ ]:
data.head()

In [ ]:
X = data.iloc[:, 0:30]
X.head()

In [ ]:
y = data['Class']
y.head()

In [ ]:
corr = X.corr().abs()
lower_right_ones = np.tril(np.ones(corr.shape, dtype='bool'), k=-1)
correlations = corr.where(lower_right_ones)
correlations

In [ ]:
plt.figure(figsize=(30,30))
sns.heatmap(correlations, annot=True, cmap='RdBu_r', fmt= '.2f', vmax=1, vmin=-1)
plt.xticks(rotation=60)

No significant correlation found, as seen in above plot.

## Preprocessing

#### Splitting the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2, test_size=0.20)

## Using SMOTE

In [ ]:
print("Data transformation -")
print("Normal data distribution: {}".format(Counter(y)))
X_smote, y_smote = SMOTE().fit_sample(X, y)
print("SMOTE data distribution: {}".format(Counter(y_smote)))

### Build model with SMOTE imblearn

In [ ]:
smote_pipeline = make_pipeline_imb(SMOTE(random_state=4), RandomForestClassifier(random_state=42))

In [ ]:
SMOTE_model = smote_pipeline.fit(X_train, y_train)
SMOTE_prediction = SMOTE_model.predict(X_test)

In [ ]:
def print_results(headline, true_value, pred):
    print(headline)
    print("accuracy: {}".format(accuracy_score(true_value, pred)))
    print("precision: {}".format(precision_score(true_value, pred)))
    print("recall: {}".format(recall_score(true_value, pred)))
    print("f2: {}".format(fbeta_score(true_value, pred, beta=2)))


## Evaluating the model SMOTE + Random Forest

In [ ]:
print("Confusion Matrix: ")
print(confusion_matrix(y_test, SMOTE_prediction))

print('\nSMOTE Pipeline Score {}'.format(smote_pipeline.score(X_test, y_test)))

print_results("\nSMOTE + RandomForest classification", y_test, SMOTE_prediction)

In [ ]:
# Compute predicted probabilities: y_pred_prob
y_pred_prob = smote_pipeline.predict_proba(X_test)[:,1]

# Generate precision recall curve values: precision, recall, thresholds
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_prob)

# Plot ROC curve
plt.plot(precision, recall)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision Recall Curve')
plt.show()